# Classification: End-to-end

-----

Kaggle StumbleUpon Competition
==================
https://www.kaggle.com/c/stumbleupon

** Competition **: 
1. Some web pages, such as news articles or seasonal recipes, are only relevant for a short period of time. Others continue to be important for a long time.
2. The goal is to identify pages which pages will be relevant for a short span of time, and which will be relevant for a long span on time and are thus considered "evergreen". 

** Evaluation **: Area under the curve (AUC) 


-----

Import Python Modules 
=================

In [14]:
# quick hack to fix import path
# import sys; sys.path.append('/Users/julianalverio/code/conda/envs/sac/lib/python3.6/site-packages/')

# data manipulation
import pandas as pd
import numpy as np

# plots
%matplotlib inline
import random
import matplotlib
import matplotlib.pyplot as plt
import pylab as pl

# classification algorithms
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm

# dimensionality reduction
from sklearn.decomposition import PCA

# cross-validation
from sklearn.model_selection import train_test_split
from sklearn import model_selection


# model evaluation
from sklearn.metrics import roc_auc_score

import warnings
warnings.filterwarnings("ignore")

import os
os.chdir(os.path.join("..", "data"))

# 1. Data Import

In [15]:
data = pd.read_table("stumbleupon/train.tsv", sep= "\t")

## Variable descriptions:

1. **url**: Url of the webpage to be classified
2. **urlid**: StumbleUpon's unique identifier for each url
3. **boilerplate**: Boilerplate text
4. **alchemy_category**:	Alchemy category
5. **alchemy_category_score**:	Alchemy category score
6. **avglinksize**:	Average number of words in each link
7. **commonLinkRatio_1**:	# of links sharing at least 1 word with 1 other links / # of links
8. **commonLinkRatio_2**:	# of links sharing at least 1 word with 2 other links / # of links
9. **commonLinkRatio_3**:	# of links sharing at least 1 word with 3 other links / # of links
10. **commonLinkRatio_4**:	# of links sharing at least 1 word with 4 other links / # of links
11. **compression_ratio**:	Compression achieved on this page via gzip (measure of redundancy)
12. **embed_ratio**: Count of number of "embed" usage
13. **frameBased**: A page is frame-based (1) if it has no body markup but have a frameset markup
14. **frameTagRatio**: Ratio of iframe markups over total number of markups
15. **hasDomainLink**:	True (1) if it contains an "a" with an url with domain
16. **html_ratio**:	Ratio of tags vs text in the page
17. **image_ratio**: Ratio of "img" tags vs text in the page
18. **is_news**: True (1) if StumbleUpon's news classifier determines that this webpage is news
19. **lengthyLinkDomain**: True (1) if at least 3 "a"'s text contains more than 30 alphanumeric characters
20. **linkwordscore**: Percentage of words on the page that are in hyperlink's text
21. **news_front_page**: True (1) if StumbleUpon's news classifier determines that this webpage is front-page news
22. **non_markup_alphanum_characters**:	integer	Page's text's number of alphanumeric characters
23. **numberOfLinks**: Number of "a"  markups
24. **numwords_in_url**: Number of words in url
25. **parametrizedLinkRatio**: A link is parametrized if it's url contains parameters  or has an attached onClick event
26. **spelling_errors_ratio**: Ratio of words not found in wiki (considered to be a spelling mistake)
27. **label**: User-determined label. Either evergreen (1) or non-evergreen (0)


## Feature Engineering

In [16]:
'''
No Good
url -- object type, the literal url
urlid -- uid for the url, drop it
avglinksize -- huge spread, mostly around 1 or two. This would need to be hella binned carefully. I think useless from Madhav's results
frameBased -- 0 everywhere, garbage
is_news -- 1 everywhere

TEXT
boilerplate -- text feature

NUMERICAL
alchemy_category -- business, health, sports, etc.
alchemy_category_score -- 0 to 1 STRING with LOTS of '?' values
commonlinkratio -- cont value from 0 to 1 that works nicely, needs binning. Nice gaussian on 1, moves to left with increasing idx
compression_ratio -- continuous value with huge bimodal spread -- needs to be binned carefully
embed_ratio -- perfect bimodal spread -- two humps. -1 and 0.25
frameTagRatio -- 0 to 0.44 cont, spread leans left -- it is half a gaussian
hasDomainLink -- 0/1 categorical, almost all 0
html_ratio -- cont 0 to 0.7, gaussian spread
image_ratio -- spread -1 to 113, but almost exclusively right at 0
lengthy-link domain -- categorical 0/1, good spread
linkwordscore -- 0 to 100 good spread
news_front_page -- STRING, ? values, 0-1 categorical, almost all 0
non_markup_alphnum_characters -- huge spread, highly concentrated near 0
numberoflinks -- huge spread, mostly close to 0
numwordsinurl -- good spread, 0 to 25, has a long upper tail
Parametrizedlinkratio -- 0 to 1 continuous, skewed toward 0 -- right half of gaussian
spelling_errors_ratio -- 0 to 1, heavy leaning to 0


Entropy measures on each feature

[(-0.0017029098026004608, 'avglinksize'),
 (-0.0015477640384510272, 'image_ratio'),
 (-0.0013366491854696072, 'numberOfLinks'),
 (-0.0012422148274934264, 'urlid'),
 (-0.0012276386004722584, 'non_markup_alphanum_characters'),
 (-0.0003632131231506852, 'hasDomainLink'),
 (-1.6445368175466157e-05, 'compression_ratio'),
 (0, 'alchemy_category_score'),
 (0, 'boilerplate'),
 (0, 'commonLinkRatio_1'),
 (0, 'commonLinkRatio_2'),
 (0, 'commonLinkRatio_3'),
 (0, 'commonLinkRatio_4'),
 (0, 'frameBased'),
 (0, 'is_news'),
 (0, 'label'),
 (0, 'news_front_page'),
 (0, 'url'),
 (0.00016331922818457745, 'lengthyLinkDomain'),
 (0.0007063846400122697, 'html_ratio'),
 (0.0011190978216005787, 'embed_ratio'),
 (0.0020862547468744053, 'spelling_errors_ratio'),
 (0.002983052198938685, 'parametrizedLinkRatio'),
 (0.004883582510772921, 'numwords_in_url'),
 (0.016376170421172676, 'linkwordscore'),
 (0.016942353001673127, 'frameTagRatio'),
 (0.040352206434091764, 'alchemy_category')]

'''

# Alchemy category, converting to one-hots
df = data['alchemy_category']   # 2K ? values
one_hots = pd.get_dummies(data['alchemy_category'])
df = one_hots
rename_dict = {'?': 'alchemy_cat_?'}
df = df.rename(columns=rename_dict)

# FrameTagRatio, leaving as continuous number
df_var = data['frameTagRatio']
df['frame_tag_ratio'] = df_var



# link word score, 0-100 gaussian, keeping continuous
df['link_word_score'] = data['linkwordscore']


# alchemy category score, with replacing missing values with random
df_var = data['alchemy_category_score']
df_var_temp = df_var.apply(lambda x: np.random.random() if x == '?' else float(x)).astype('float32')
df['alchemy_category_score'] = df_var_temp


# num word in url -- discrete 0-25 to custom binning from looking at the histogram
df_var = data['numwords_in_url']
bins = [0, 6, 8, 13, 25]
df_var_temp = pd.cut(x=df_var, bins=bins, right=True, labels=['num_words_url_bin_0', 'num_words_url_bin_1', 'num_words_url_bin_2', 'num_words_url_bin_3'])
dummies = pd.get_dummies(df_var_temp)
df = pd.concat([df, dummies], axis=1)


# parameterized_link_ratio -- leaving as continuous, right-half gaussian
df['parameterized_link_ratio'] = data['parametrizedLinkRatio']

# spelling errors ratio -- leaving as continuous
df['spelling_errors_ratio'] = data['spelling_errors_ratio']

# embed_ratio -- bimodal continuous binned into 2 bins
df_var = pd.DataFrame(data['embed_ratio'])
df_var = df_var['embed_ratio'].apply(lambda x: 1 if x > -1 else 0)
dummies = pd.get_dummies(df_var)
rename = {0: 'embed_ratio_0', 1: 'embed_ratio_1'}
dummies = dummies.rename(columns=rename)
df = pd.concat([df, dummies], axis=1)


# html_ratio -- leaving continuous
df['html_ratio'] = data['html_ratio']

# lengthy_link_domain
df_var = pd.get_dummies(data['lengthyLinkDomain'])
rename = {0: 'lengthy_link_domain_0', 1: 'lengthy_link_domain_1'}
df_var = df_var.rename(columns=rename)
df = pd.concat([df, df_var], axis=1)

df['labels'] = data['label']


In [17]:
train, val = train_test_split(df, test_size=0.5, train_size=0.5, random_state=234)
val, test = train_test_split(val, test_size=0.5, train_size=0.5, random_state= 675)
train_labels = train['labels']
train = train.drop(['labels'], axis=1, inplace=False)
val_labels = val['labels']
val = val.drop(['labels'], axis=1, inplace=False)
test_labels = test['labels']
test = test.drop(['labels'], axis=1, inplace=False)

In [18]:
def logistic_search(budget):
    ret = []
    for i in range(budget):
        seed = random.randrange(500)
        rand_C =    random.choice([0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000])
        rand_fit_intercept = random.choice([True, False])
        rand_penalty = random.choice(['l1', 'l2'])
        model = LogisticRegression(random_state=seed, C = rand_C, fit_intercept = rand_fit_intercept, penalty=rand_penalty)
        model.fit(train, train_labels)
        preds = model.predict_proba(val)[:,1]
        score = roc_auc_score(val_labels, preds)
        ret.append((score, model))
    return list(reversed(sorted(ret, key=lambda x: x[0])))

In [19]:
# Best logistic regression: AUC=0.71
seed = 61
C = 1
fit_intercept = False
penalty = 'l1'
model = LogisticRegression(random_state=seed, C=C, fit_intercept=fit_intercept, penalty=penalty)
model.fit(train, train_labels)
preds = model.predict_proba(val)[:,1]
score = roc_auc_score(val_labels, preds)
print(score)

0.7186933252833297


In [20]:
logistic_search(100)

[(0.7187015156939969,
  LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=False,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l1', random_state=475, solver='warn',
            tol=0.0001, verbose=0, warm_start=False)),
 (0.7186816246966623,
  LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=False,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l1', random_state=386, solver='warn',
            tol=0.0001, verbose=0, warm_start=False)),
 (0.7186769444619953,
  LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=False,
            intercept_scaling=1, max_iter=100, multi_class='warn',
            n_jobs=None, penalty='l1', random_state=84, solver='warn',
            tol=0.0001, verbose=0, warm_start=False)),
 (0.7186746043446618,
  LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=False,
            interce

In [21]:
def rbdt_search(budget):
    ret = []
    for i in range(budget):
        # choose a set of random hyper-parameters
        seed = random.randrange(500)
        ccp_alpha = random.choice([0.0, 0.1])
        learning_rate = random.choice([0.001, 0.01, 0.1])
        n_estimators = random.choice([10, 30, 80, 160, 300])
        max_depth = random.choice([1,3,5,10])
        model = GradientBoostingClassifier(random_state=seed, learning_rate=learning_rate, n_estimators=n_estimators, max_depth = max_depth)
        model.fit(train, train_labels)
        preds = model.predict_proba(val)[:,1]
        score = roc_auc_score(val_labels, preds)
        ret.append((score, model))
    return list(reversed(sorted(ret, key=lambda x: x[0])))

In [22]:
# Best Gradient Boosted Decision Tree: AUC=0.73
learning_rate = 0.01
max_depth = 3
n_estimators = 300
random_state = 239
model = GradientBoostingClassifier(random_state=seed, learning_rate=learning_rate, n_estimators=n_estimators, max_depth = max_depth)
model.fit(train, train_labels)
preds = model.predict_proba(val)[:,1]
score = roc_auc_score(val_labels, preds)
print(score)

0.7345142735456758


In [23]:
rbdt_search(100)

[(0.7345142735456758,
  GradientBoostingClassifier(criterion='friedman_mse', init=None,
                learning_rate=0.01, loss='deviance', max_depth=3,
                max_features=None, max_leaf_nodes=None,
                min_impurity_decrease=0.0, min_impurity_split=None,
                min_samples_leaf=1, min_samples_split=2,
                min_weight_fraction_leaf=0.0, n_estimators=300,
                n_iter_no_change=None, presort='auto', random_state=211,
                subsample=1.0, tol=0.0001, validation_fraction=0.1,
                verbose=0, warm_start=False)),
 (0.7345142735456758,
  GradientBoostingClassifier(criterion='friedman_mse', init=None,
                learning_rate=0.01, loss='deviance', max_depth=3,
                max_features=None, max_leaf_nodes=None,
                min_impurity_decrease=0.0, min_impurity_split=None,
                min_samples_leaf=1, min_samples_split=2,
                min_weight_fraction_leaf=0.0, n_estimators=300,
              

In [ ]:
def svm_search(budget):
    ret = []
    for i in range(budget):
        # choose a set of random hyper-parameters
        seed = random.randrange(500)
        C = random.choice([0.0001, 0.001, 0.01, 0.1, 1.0])
        kernel = random.choice(['linear', 'poly', 'rbf', 'sigmoid'])
        print ("running for ", C, kernel)
        model = svm.SVC(random_state=seed, C=C, kernel=kernel, probability=True)
        model.fit(train, train_labels)
        preds = model.predict_proba(val)[:,1]
        score = roc_auc_score(val_labels, preds)
        ret.append((score, model))
        print (score, model)
    return list(reversed(sorted(ret, key=lambda x: x[0])))


In [ ]:
svm_search(20)

running for  0.0001 linear
0.6091231814363173 SVC(C=0.0001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=True, random_state=131,
  shrinking=True, tol=0.001, verbose=False)
running for  0.1 sigmoid
0.6006542968064419 SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='sigmoid', max_iter=-1, probability=True, random_state=91,
  shrinking=True, tol=0.001, verbose=False)
running for  0.1 linear
0.7016912027969083 SVC(C=0.1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=True, random_state=184,
  shrinking=True, tol=0.001, verbose=False)
running for  0.0001 linear
0.6091208413189837 SVC(C=0.0001, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamm

In [ ]:
# Best SVM: AUC=0.7
C = 0.1
kernel = 'linear'
seed = 184
model = svm.SVC(random_state=seed, C=C, kernel=kernel, probability=True)
model.fit(train, train_labels)
preds = model.predict_proba(val)[:,1]
score = roc_auc_score(val_labels, preds)
print(score)

** Note **: Scikit-learn training algorithms do not accept categorical features and hence they need to be converted to numeric or binary before fitting the model 

We did really well for only using numerical features, but there's much more to be done! Dimensionality reduction, text processing, and more. Stay tuned for next week's natural language processing (NLP) theme.
===========